In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import hashing_trick
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random


import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [3]:
wynik1 = round(random.randint(200, 2000), -2)
wynik2 = round(random.randint(100, 1500), -2)
wynik3 = round(random.randint(20, 200), -1)
num_epochs = round(random.randint(10, 40), 0)

batchsize = 500

print(wynik1, wynik2, wynik3, num_epochs)

1300 200 130 24


In [4]:
metrics = [keras.metrics.TruePositives(name='tp'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.TrueNegatives(name='tn'),
           keras.metrics.FalseNegatives(name='fn'),
           keras.metrics.BinaryAccuracy(name='accuracy'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.AUC(name='auc')]

In [5]:
t=time.time()

In [6]:
dset = pd.read_csv('data/probkauczaca2.csv', sep=';',encoding='utf-8')

Y = dset['Default'].values

In [7]:
vartype =  pd.read_excel('data/rodzaje_zmiennych.xlsx')

In [8]:
columnX=[]
X_binary = []
X_discrete = []
X_text_cat = []
for i in range(vartype.shape[1]):
    if vartype.iloc[0][i] == "numeric_continuous":
        columnX.append(dset.columns[i])
    if vartype.iloc[0][i] == "numeric_binary":
        X_binary.append(dset.columns[i])
    if vartype.iloc[0][i] == "numeric_discrete":
        X_discrete.append(dset.columns[i])
    if vartype.iloc[0][i] == "text_categorical":
        X_text_cat.append(dset.columns[i])

print(columnX)
print(X_binary)
print(X_discrete)
print(X_text_cat)


['Przychody', 'Suma_bilansowa', 'CNT_WPPLN', 'CNT_OWPLN', 'CNT_WKPLN', 'OWPRC', 'WKPRC', 'MARZA', 'Admin', 'PROWIZJA', 'Kapital']
['Default', 'Czy_cesja', 'Czy_leasing_zwrotny', 'Wynik_sprawdzen_KRD']
['Liczba_prawcownikow', 'Rok_produkcji', 'Okres_umowy', 'CzasDzialalnosciMce']
['OBT_Type2', 'OBT_IFNEW', 'Czy_autoryzowany_dostawca', 'Waluta', 'Rodzaj_harmonogramu', 'Rodzaj_oprocentowania', 'Dodatkowe_zabezpieczenie_transakcji', 'Kanal_pozyskania', 'Oddzial', 'Rodzaj_umowy', 'Procedura', 'PKD_2007', 'PROW_Z_MARZY', 'forma_prawna', 'Grupowanie_form_prawnych']


In [9]:
X = dset.loc[:, columnX].astype(float)
X['Kapital'] = X['Kapital'].fillna(0)

In [10]:
scaler = StandardScaler()

# Standaryzacja wszystkich zmiennych w dataframe X
X_standardized = scaler.fit_transform(X)
# Tworzenie nowego dataframe z zstandaryzowanymi danymi
X_standardized_df = pd.DataFrame(X_standardized, columns=X.columns)

In [11]:
# Tworzenie obiektu MinMaxScaler
scaler_minmax = MinMaxScaler()

# Normalizacja min-max dla zmiennych z X_discrete
for column_name in X_discrete:
    dset[column_name] = scaler_minmax.fit_transform(dset[[column_name]])

In [12]:
X_bin = dset.loc[:,['Czy_cesja', 'Czy_leasing_zwrotny', 'Wynik_sprawdzen_KRD']].astype(str)
X_yesno = pd.DataFrame(X_bin['Czy_cesja'].eq('1').mul(1))
#X_yesno = pd.DataFrame(X_binary['OBT_IFNEW'].eq('New').mul(1))
#X_yesno['Czy_aut_dostawca'] = X_binary['Czy_autoryzowany_dostawca'].eq('T').mul(1)
#X_yesno['Waluta'] = X_binary['Waluta'].eq('PLN').mul(1)
#X_yesno['Rodzaj_harm'] = X_binary['Rodzaj_harmonogramu'].eq('liniowy').mul(1)
#X_yesno['Rodzaj_oproc'] = X_binary['Rodzaj_oprocentowania'].eq('V').mul(1)
##X_yesno['Dodat_zabezp_trans'] = X_binary['Dodatkowe_zabezpieczenie_transakcji'].eq('WEKSEL').mul(1)
#X_yesno['Kanal_pozysk'] = X_binary['Kanal_pozyskania'].eq('Porednik').mul(1)
#X_yesno['Cesja'] = X_bin['Czy_cesja'].eq('1').mul(1)
#X_yesno['Auto_luks'] = X_binary['Auto_luksusowe'].eq('1').mul(1)
#X_yesno['RSEUO'] = X_binary['RS_EUO'].eq('1').mul(1)
#X_yesno['BiuroWirtK'] = X_binary['BiuroWirtualneKlient'].eq('1').mul(1)
#X_yesno['BiuroWirtD'] = X_binary['BiuroWirtualneDostawca'].eq('1').mul(1)
#X_yesno['BiuroWirtP'] = X_binary['BiuroWirtualnePosrednik'].eq('1').mul(1)
X_yesno['Czy_leas_zwr'] = X_bin['Czy_leasing_zwrotny'].astype(float)
X_yesno['Wyn_spraw_KRD'] = X_bin['Wynik_sprawdzen_KRD'].astype(float)

In [13]:
# ----- ładowanie danych głównego kodu PKD ------
# w tym przypadku wykorzystywane są tylko fragmenty PKD /sekcja, dział, grupa/

#dodać zmienną x_text_kat - zrobione

#zaminić dset na zmienną X_text_cat - zrobić!

#X_text_cat['kodpkd'] = X_text_cat['PKD_2007'].str.get(0)+X_text_cat['PKD_2007'].str.get(1)+X_text_cat['PKD_2007'].str.get(2)
dset['kodpkd'] = dset['PKD_2007'].str.get(0)+dset['PKD_2007'].str.get(1)+dset['PKD_2007'].str.get(2)


In [14]:
# ---- ładowanie danych rodzaj umowy; oddział; grupowanie form prawnych; branza; OBT_Type2 (czyli rodzaj przedmiotu lisingu)
#zaminić dset na zmienną X_text_cat - zrobić
dset['rodzumowy'] = dset['Rodzaj_umowy'].astype(str)
dset['oddzial1'] = dset['Oddzial'].astype(str)
dset['oddzial1'] = dset['oddzial1'].str.replace(' ', '')
dset['oddzial1'] = dset['oddzial1'].str.replace('-', '')
dset['oddzial1'] = dset['oddzial1'].str.strip()
dset['grupowanie'] = dset['Grupowanie_form_prawnych'].astype(str)
dset['branza1'] = dset['Branza'].astype(str)
dset['OBT_Type2'] = dset['OBT_Type2'].str.replace(' ', '')
dset['OBT_Type2'] = dset['OBT_Type2'].str.replace('&', '')

#X_text_cat['rodzumowy'] = X_text_cat['Rodzaj_umowy'].astype(str)
#X_text_cat['oddzial1'] = X_text_cat['Oddzial'].astype(str)
#X_text_cat['oddzial1'] = X_text_cat['oddzial1'].str.replace(' ', '')
#X_text_cat['oddzial1'] = X_text_cat['oddzial1'].str.replace('-', '')
#X_text_cat['oddzial1'] = X_text_cat['oddzial1'].str.strip()
#X_text_cat['grupowanie'] = X_text_cat['Grupowanie_form_prawnych'].astype(str)
#X_text_cat['branza1'] = X_text_cat['Branza'].astype(str)
#X_text_cat['OBT_Type2'] = X_text_cat['OBT_Type2'].str.replace(' ', '')
#X_text_cat['OBT_Type2'] = X_text_cat['OBT_Type2'].str.replace('&', '')

In [15]:
# ---- ładowanie danych z bazy danych z tabeli cechy scoringowe
#X_scor = pd.DataFrame(dset.iloc[:, 35:92].values.astype(int)) # 57 kolumn

In [16]:
def one_hot_encode(data, column, size):
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(data[[column]])
    encoded_df = pd.DataFrame(encoded_data, columns=[f'{column}_{i}' for i in range(encoded_data.shape[1])])
    return encoded_df

def apply_hash(data, column, size):
    encoded_hash = [hash(str(d)) % size for d in data[column]]
    encoded_hash_df = pd.DataFrame(encoded_hash, columns=[f'{column}_hash'])
    return encoded_hash_df

# One-Hot Encoding dla 'pkd'
pkd_size = 272
X_pkd = one_hot_encode(dset, 'kodpkd', pkd_size)

# Hashing Trick dla 'pkd'
X_pkd_hash = apply_hash(dset, 'kodpkd', pkd_size)

# One-Hot Encoding dla 'rodzumowy'
rumowy_size = 4
X_rumowy = one_hot_encode(dset, 'rodzumowy', rumowy_size)

# Hashing Trick dla 'rodzumowy'
X_rumowy_hash = apply_hash(dset, 'rodzumowy', rumowy_size)

# One-Hot Encoding dla 'grupowanie'
grupowanie_size = 4
X_grupowanie = one_hot_encode(dset, 'grupowanie', grupowanie_size)

# Hashing Trick dla 'grupowanie'
X_grupowanie_hash = apply_hash(dset, 'grupowanie', grupowanie_size)

# One-Hot Encoding dla 'oddzial1'
oddzial1_size = 24
X_oddzial1 = one_hot_encode(dset, 'oddzial1', oddzial1_size)

# Hashing Trick dla 'oddzial1'
X_oddzial1_hash = apply_hash(dset, 'oddzial1', oddzial1_size)

# One-Hot Encoding dla 'branza1'
branza1_size = 7
X_branza1 = one_hot_encode(dset, 'branza1', branza1_size)

# Hashing Trick dla 'branza1'
X_branza1_hash = apply_hash(dset, 'branza1', branza1_size)

# One-Hot Encoding dla 'OBT_Type2'
objlise_size = 7
X_objlise = one_hot_encode(dset, 'OBT_Type2', objlise_size)

# Hashing Trick dla 'OBT_Type2'
X_objlise_hash = apply_hash(dset, 'OBT_Type2', objlise_size)

In [17]:
tdset = pd.read_csv('data/probkatestowa2.csv', sep=';',encoding='utf-8')
del tdset['Data_bilansowa']

tY = tdset['Default'].values

In [18]:
#print(tdset)

tX = tdset.loc[:, columnX].astype(float)
# usuwa Null z 3 kolumn

tX['Kapital'] = tX['Kapital'].fillna(0)

tscaler = StandardScaler()

# Standaryzacja wszystkich zmiennych w dataframe X
tX_standardized = scaler.fit_transform(tX)
# Tworzenie nowego dataframe z zstandaryzowanymi danymi
tX_standardized_df = pd.DataFrame(tX_standardized, columns=tX.columns)

# Tworzenie obiektu MinMaxScaler
tscaler_minmax = MinMaxScaler()

# Normalizacja min-max dla zmiennych z X_discrete
for column_name in X_discrete:
    tdset[column_name] = scaler_minmax.fit_transform(tdset[[column_name]])

tX_bin = tdset.loc[:,['Czy_cesja', 'Czy_leasing_zwrotny', 'Wynik_sprawdzen_KRD']].astype(str)

tX_yesno = pd.DataFrame(tX_bin['Czy_cesja'].eq('1').mul(1))
#tX_yesno['Czy_aut_dostawca'] = tX_binary['Czy_autoryzowany_dostawca'].eq('T').mul(1)
#tX_yesno['Waluta'] = tX_binary['Waluta'].eq('PLN').mul(1)
#tX_yesno['Rodzaj_harm'] = tX_binary['Rodzaj_harmonogramu'].eq('liniowy').mul(1)
#tX_yesno['Rodzaj_oproc'] = tX_binary['Rodzaj_oprocentowania'].eq('V').mul(1)
#tX_yesno['Dodat_zabezp_trans'] = tX_binary['Dodatkowe_zabezpieczenie_transakcji'].eq('WEKSEL').mul(1)
#tX_yesno['Kanal_pozysk'] = tX_binary['Kanal_pozyskania'].eq('Porednik').mul(1)
#tX_yesno['Cesja'] = tX_bin['Czy_cesja'].eq('1').mul(1)
#tX_yesno['Auto_luks'] = tX_binary['Auto_luksusowe'].eq('1').mul(1)
#tX_yesno['RSEUO'] = tX_binary['RS_EUO'].eq('1').mul(1)
#tX_yesno['BiuroWirtK'] = tX_binary['BiuroWirtualneKlient'].eq('1').mul(1)
#tX_yesno['BiuroWirtD'] = tX_binary['BiuroWirtualneDostawca'].eq('1').mul(1)
#tX_yesno['BiuroWirtP'] = tX_binary['BiuroWirtualnePosrednik'].eq('1').mul(1)
tX_yesno['Czy_leas_zwr'] = tX_bin['Czy_leasing_zwrotny'].astype(float)
tX_yesno['Wyn_spraw_KRD'] = tX_bin['Wynik_sprawdzen_KRD'].astype(float)
tdset['kodpkd'] = tdset['PKD_2007'].str.get(0)+tdset['PKD_2007'].str.get(1)+tdset['PKD_2007'].str.get(2)

tdset['rodzumowy'] = tdset['Rodzaj_umowy'].astype(str)
tdset['oddzial1'] = tdset['Oddzial'].astype(str)
tdset['oddzial1'] = tdset['oddzial1'].str.replace(' ', '')
tdset['oddzial1'] = tdset['oddzial1'].str.replace('-', '')
tdset['oddzial1'] = tdset['oddzial1'].str.strip()
tdset['grupowanie'] = tdset['Grupowanie_form_prawnych'].astype(str)
tdset['branza1'] = tdset['Branza'].astype(str)
tdset['OBT_Type2'] = tdset['OBT_Type2'].str.replace(' ', '')
tdset['OBT_Type2'] = tdset['OBT_Type2'].str.replace('&', '')

In [19]:
def one_hot_encode(data, column, size):
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(data[[column]])
    encoded_df = pd.DataFrame(encoded_data, columns=[f'{column}_{i}' for i in range(encoded_data.shape[1])])
    return encoded_df

def apply_hash(data, column, size):
    encoded_hash = [hash(str(d)) % size for d in data[column]]
    encoded_hash_df = pd.DataFrame(encoded_hash, columns=[f'{column}_hash'])
    return encoded_hash_df

# One-Hot Encoding dla 'pkd'
tX_pkd = one_hot_encode(dset, 'kodpkd', pkd_size)

# Hashing Trick dla 'pkd'
tX_pkd_hash = apply_hash(dset, 'kodpkd', pkd_size)

# One-Hot Encoding dla 'rodzumowy'
tX_rumowy = one_hot_encode(dset, 'rodzumowy', rumowy_size)

# Hashing Trick dla 'rodzumowy'
tX_rumowy_hash = apply_hash(dset, 'rodzumowy', rumowy_size)

# One-Hot Encoding dla 'grupowanie'
tX_grupowanie = one_hot_encode(dset, 'grupowanie', grupowanie_size)

# Hashing Trick dla 'grupowanie'
tX_grupowanie_hash = apply_hash(dset, 'grupowanie', grupowanie_size)

# One-Hot Encoding dla 'oddzial1'
tX_oddzial1 = one_hot_encode(dset, 'oddzial1', oddzial1_size)

# Hashing Trick dla 'oddzial1'
tX_oddzial1_hash = apply_hash(dset, 'oddzial1', oddzial1_size)

# One-Hot Encoding dla 'branza1'
tX_branza1 = one_hot_encode(dset, 'branza1', branza1_size)

# Hashing Trick dla 'branza1'
tX_branza1_hash = apply_hash(dset, 'branza1', branza1_size)

# One-Hot Encoding dla 'OBT_Type2'
tX_objlise = one_hot_encode(dset, 'OBT_Type2', objlise_size)

# Hashing Trick dla 'OBT_Type2'
tX_objlise_hash = apply_hash(dset, 'OBT_Type2', objlise_size)

In [20]:
#------------------------ Model sztucznej sieci neuronowej (ANN)-----------------------------------------------------------


inputpkd = Input(shape=(1,), name='in_pkd')
pkd = Embedding(pkd_size, 10, input_length=1)(inputpkd)
pkd_out = Flatten()(pkd)
pkd_out = Dense(200, activation='relu')(pkd_out)

inputrumowy = Input(shape=(1,), name='in_rodzaj_umowy')
rumowy = Embedding(rumowy_size, 3, input_length=1)(inputrumowy)
rumowy_out = Flatten()(rumowy)
rumowy_out = Dense(100, activation='relu')(rumowy_out)

inputgrupowanie = Input(shape=(1,), name='in_grupowanie')
grupowanie = Embedding(grupowanie_size, 3, input_length=1)(inputgrupowanie)
grupowanie_out = Flatten()(grupowanie)
grupowanie_out = Dense(100, activation='relu')(grupowanie_out)

inputoddzial1 = Input(shape=(1,), name='in_oddzial1')
oddzial1 = Embedding(oddzial1_size, 10, input_length=1)(inputoddzial1)
oddzial1_out = Flatten()(oddzial1)
oddzial1_out = Dense(100, activation='relu')(oddzial1_out)

inputbranza1 = Input(shape=(1,), name='in_branza1')
branza1 = Embedding(branza1_size, 4, input_length=1)(inputbranza1)
branza1_out = Flatten()(branza1)
branza1_out = Dense(100, activation='relu')(branza1_out)

inputobjlise = Input(shape=(1,), name='in_lise_object')
objlise = Embedding(objlise_size, 4, input_length=1)(inputobjlise)
objlise_out = Flatten()(objlise)
objlise_out = Dense(100, activation='relu')(objlise_out)

inputnumbers = Input(shape=(11,), name='in_continous')
innum = Dense(200, activation="relu")(inputnumbers)
innum = BatchNormalization()(innum)
num_out = Dense(100, activation="relu")(innum)

inputyesno = Input(shape=(3,), name='in_bool')
inyesno = Dense(15, activation="relu")(inputyesno)
yesno_out = Dense(10, activation="relu")(inyesno)

#inputscor = Input(shape=(57,), name='in_scoring')
#scor = Embedding(scor_size, 10, input_length=57)(inputscor)
#scor_out = Flatten()(scor)
#scor_out = Dense(400, activation='relu')(scor_out)

#inputcat = Input(shape=(322,), name='in_categorical')
#catcode = Dense(1000, activation="relu")(inputcat)
#cat_out = Dense(100, activation='relu')(catcode)

In [21]:
merge = concatenate([pkd_out, num_out,rumowy_out,grupowanie_out, yesno_out, oddzial1_out, branza1_out, objlise_out], axis=-1)

hidden = Dense(wynik1, activation='relu')(merge)
hidden = BatchNormalization()(hidden)
hidden = Dense(wynik2, activation='relu')(hidden)
hidden = Dense(wynik3, activation='relu')(hidden)
output = Dense(1, activation='sigmoid')(hidden)

model = Model(inputs=[inputpkd,  inputnumbers, inputrumowy, inputgrupowanie, inputyesno, inputoddzial1,  inputbranza1, inputobjlise], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=metrics)

In [22]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 in_pkd (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 in_continous (InputLayer)   [(None, 11)]                 0         []                            
                                                                                                  
 in_rodzaj_umowy (InputLaye  [(None, 1)]                  0         []                            
 r)                                                                                               
                                                                                                  
 in_grupowanie (InputLayer)  [(None, 1)]                  0         []                        

                                                                    ]                             
                                                                                                  
 dense_12 (Dense)            (None, 130)                  26130     ['dense_11[0][0]']            
                                                                                                  
 dense_13 (Dense)            (None, 1)                    131       ['dense_12[0][0]']            
                                                                                                  
Total params: 1377621 (5.26 MB)
Trainable params: 1374621 (5.24 MB)
Non-trainable params: 3000 (11.72 KB)
__________________________________________________________________________________________________
None


In [23]:
# Sprawdź kształt danych wejściowych
print(X.shape)

# Sprawdź wartość argumentu `input_shape` dla warstwy `Dense`
print(model.layers[0].input_shape)

# Sprawdź wersję TensorFlow
print(tf.__version__)

(10000, 11)
[(None, 1)]
2.13.0


In [29]:
# Definiowanie tablic do połączenia
arrays = [X, X_pkd, X_pkd_hash, X_rumowy, X_rumowy_hash, X_grupowanie, X_grupowanie_hash, X_oddzial1, X_oddzial1_hash, X_branza1, X_branza1_hash, X_objlise, X_objlise_hash]

# Łączenie tablic wzdłuż drugiej osi
X_combined = np.concatenate(arrays, axis=1)

tX_resized = np.resize(tX, (X.shape[0], tX.shape[1]))
# Powtórzenie tego samego procesu dla tablic tX
t_arrays = [tX, tX_pkd, tX_pkd_hash, tX_rumowy, tX_rumowy_hash, tX_grupowanie, tX_grupowanie_hash, tX_oddzial1, tX_oddzial1_hash, tX_branza1, tX_branza1_hash, tX_objlise, tX_objlise_hash]
tX_combined = np.concatenate(t_arrays, axis=1)

# One-Hot Encoding
encoder = OneHotEncoder()
encoder.fit(X_combined)
X_encoded = encoder.transform(X_combined)
tX_encoded = encoder.transform(tX_combined)

# Podział danych na zbiór treningowy i walidacyjny
X_train, X_val, Y_train, Y_val = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)

# Tworzenie modelu
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_encoded.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Kompilacja modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
start_time = time.time()
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=num_epochs, batch_size=batchsize)

# Zapis modelu
model.save_weights("model3_weights.h5", overwrite=True)
model.save('model3_structure.h5', overwrite=True)

# Wyświetlenie czasu symulacji
print("Czas symulacji:", time.time() - start_time)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3436 and the array at index 1 has size 10000

In [ ]:
model.save_weights("model3_weights.h5", overwrite=True)
model.save('model3_structure.h5', overwrite=True)
print("Czas symulacji: ", time.time() - t)

In [ ]:
####użyć tylko raz, jak następny kod nie działa !!
#df = pd.DataFrame()
#file_name = 'wyniki_treningu.xlsx'
#df.to_excel(file_name, index=False)
#print(f"Plik {file_name} został utworzony.")

In [ ]:
training_results = []

for i in range(num_epochs):
    epoch_results = {
        'loss': h.history['loss'][i],
        'val_loss': h.history['val_loss'][i],
        'val_accuracy': h.history['val_accuracy'][i],
        'val_precision': h.history['val_precision'][i],
        'val_recall': h.history['val_recall'][i],
        'val_auc': h.history['val_auc'][i],
        'czas_symulacji': time.time() - t
    }

    training_results.append(epoch_results)

results_df = pd.DataFrame(training_results)

title_epochs = f"Wyniki treningu dla {num_epochs} epok"
title_1 = f"Dense 1: {wynik1}"
title_2 = f"Dense 2: {wynik2}"
title_3 = f"Dense 3: {wynik3}"

sheet_name = f'Sheet_{int(time.time())}'

with pd.ExcelWriter('wyniki_treningu.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, index=False, sheet_name=sheet_name)

    worksheet = writer.sheets[sheet_name]

    worksheet.cell(row=1, column=22, value=title_epochs)
    worksheet.cell(row=2, column=22, value=title_1)
    worksheet.cell(row=3, column=22, value=title_2)
    worksheet.cell(row=4, column=22, value=title_3)

print(results_df.head())

In [ ]:
# dodatkowe elementy (można zakomentować); rysowanie wykresu dokładności ANN; Dokładnośc ANN dla Defaultów
plt.plot(h.history['val_accuracy'])

#plt.plot(h.history['accuracy'])

#plt.plot(h.history['val_accuracy'])

#plt.title('Model accuracy')

#plt.ylabel('Accuracy')

#plt.xlabel('Epoch')

#plt.legend(['Train', 'Test'], loc='upper left')



accuracy = model.evaluate([np.array(tX_pkd),np.array(tX), np.array(tX_rumowy), np.array(tX_grupowanie), np.array(tX_yesno),
                           np.array(tX_oddzial1), np.array(tX_branza1), np.array(tX_objlise)], tY)
print('Accuracy: %f' % (accuracy[1]))

ypred = model.predict([np.array(tX_pkd),np.array(tX), np.array(tX_rumowy), np.array(tX_grupowanie), np.array(tX_yesno),
                           np.array(tX_oddzial1), np.array(tX_branza1), np.array(tX_objlise)])

count = 0
total = 0
for i in range(len(ypred)):
    if tY[i] == 1:
        total = total + 1
        if ypred[i][0] > 0.5:
            count = count + 1
print("dokładność [1]: ", count/total)

count = 0
total = 0
for i in range(len(ypred)):
    if tY[i] == 0:
        total = total + 1
        if ypred[i][0] < 0.5:
            count = count + 1
print("dokładność [0]: ", count/total)
plt.show()